In [50]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [51]:
!pip install tqdm

In [52]:
import json
with open('/Users/navnoorsingh/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [53]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [54]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
print(type(results))
results.keys()


<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [55]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [56]:
search_results = yelp_api.search_query(location = 'NY,NY',
                                      term = 'Pizza')

print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [57]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE


'Data/results_in_progress_NY_pizza.json'

In [58]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [59]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [60]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [61]:
## How many results total?
total_results = results['total']
total_results


16100

In [62]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page


20

In [63]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


805

In [64]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [65]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/805 [00:00<?, ?it/s]

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)


In [69]:
#deleting file and confirming that it is deleted.
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)


False

In [71]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 17)

In [49]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


NameError: name 'create_json_file' is not defined

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

In [ ]:
# check for duplicate results
final_df.duplicated().sum()

In [ ]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()


In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)


In [24]:
search_results['total']

16100

In [25]:
import pandas as pd
biz = pd.DataFrame(search_results["businesses"])
biz.head()


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4745,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2601,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2903,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,2268.491950
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/EBwiiS...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,864,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,1868.295512
4,KEEXuHCYhsQJ-zpJhu98bA,joes-pizza-new-york-148,Joe's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/VkqlwR...,False,https://www.yelp.com/biz/joes-pizza-new-york-1...,1763,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7547, 'longitude': -73.98696}","[delivery, pickup]",$,"{'address1': '1435 Broadway', 'address2': '', ...",+16465594878,(646) 559-4878,5345.192429


In [26]:
len(search_results["businesses"])

20

In [28]:
search_results_2 = yelp_api.search_query(location = 'NY,NY',
                             term = 'pizza',
                             offset = 20)

In [29]:
biz20 = pd.DataFrame(search_results_2['businesses'])
biz20.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,CiRj1B-gK5tRfbMkq3SBog,robertas-new-york-3,Roberta's,https://s3-media3.fl.yelpcdn.com/bphoto/__bI5t...,False,https://www.yelp.com/biz/robertas-new-york-3?a...,4,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 40.71995520720623, 'longitude': -...","[delivery, pickup]","{'address1': '196 Stanton St', 'address2': '',...",,,1479.203775,NaN
1,FhXjAc6nKLf414KxYujUHw,robertas-brooklyn-2,Roberta's,https://s3-media4.fl.yelpcdn.com/bphoto/8fY9fA...,False,https://www.yelp.com/biz/robertas-brooklyn-2?a...,3229,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.70493, 'longitude': -73.93399}","[delivery, pickup]","{'address1': '261 Moore St', 'address2': '', '...",+17184171118,(718) 417-1118,3816.592615,$$
2,bB5b8cj0yWNJvmmLiQeWGA,piccolo-trattoria-new-york-2,Piccolo Trattoria,https://s3-media4.fl.yelpcdn.com/bphoto/00Umc7...,False,https://www.yelp.com/biz/piccolo-trattoria-new...,116,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 40.7591676767848, 'longitude': -7...","[delivery, pickup]","{'address1': '455 Main St', 'address2': '', 'a...",+12127532300,(212) 753-2300,6222.244915,$
3,vfYhEpp0x-DrNjC6GSJjPQ,artichoke-basilles-pizza-new-york-2,Artichoke Basille's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/4krjwP...,False,https://www.yelp.com/biz/artichoke-basilles-pi...,4166,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7321124557704, 'longitude': -7...","[delivery, pickup]","{'address1': '321 E 14th St', 'address2': '', ...",+12122282004,(212) 228-2004,2823.002839,$
4,TWH4MjLtN1fKlF-7n6YXHg,lil-frankies-new-york,Lil Frankie's,https://s3-media3.fl.yelpcdn.com/bphoto/t0OsEV...,False,https://www.yelp.com/biz/lil-frankies-new-york...,1419,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.723625, 'longitude': -73.988555}","[delivery, pickup]","{'address1': '19 1st Ave', 'address2': '', 'ad...",+12124204900,(212) 420-4900,2006.256439,$$


In [30]:
businesses = pd.concat([biz, biz20], ignore_index = True)

display(businesses.head(3), businesses.tail(2))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4745,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2601,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2903,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,2268.491950


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
38,aiiOamKRvKcODKa5yre7Mg,rosas-pizza-new-york-6,Rosa's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/kUUoKn...,False,https://www.yelp.com/biz/rosas-pizza-new-york-...,30,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.750454562128176, 'longitude': ...","[delivery, pickup]",NaN,"{'address1': '425 7th Ave', 'address2': '', 'a...",+16469983044,(646) 998-3044,4932.383196
39,4MIHutYjIwoXsi-R8TlrSg,andrew-belluccis-pizzeria-astoria-4,Andrew Bellucci's Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/U4Gllu...,False,https://www.yelp.com/biz/andrew-belluccis-pizz...,106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.76436969988826, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '37-08 30th Ave', 'address2': '',...",+17184072497,(718) 407-2497,8287.263821
